In [ ]:
import pandas as pd
data = pd.read_csv("data_dantri_chungkhoan.csv")

In [ ]:
print(data.shape)
data.describe()

data.drop_duplicates(subset=["content"],inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)


(580, 4)


,link,title,time,content
count,580,580,580,580
unique,579,579,315,574
top,https://dantri.com.vn/kinh-doanh/con-trai-ong-...,Con trai ông Trần Đình Long đã giàu lại giàu thêm,30/10/2023,[]
freq,2,2,6,6


Trong Tiếng Anh có rất nhiều từ nối và được sử dụng rất thường xuyên như "and", "the" và "a". Khi thực hiện việc thống kê trên văn bản, những từ này sẽ mang lại rất nhiều nhiễu vì chúng xuất hiện thường xuyên hơn các từ khác. Một số pineline về NLP sẽ gắn cờ chúng là các từ dừng (stop words) - nghĩa là các từ mà bạn có thể sẽ muốn lọc ra trước khi thực hiện bất kỳ các phân tích thống kê nào. Tương tự với Tiếng Việt cũng có rất nhiều stop words, chúng ta cần phải loại bỏ chúng:

In [ ]:
# Đọc tệp vietnamesestopword.txt và lưu các từ dừng vào một danh sách
stopwords = []
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

print(f'Number of stopwords: {len(stopwords)}')
print(f'First 10 stopwords: {stopwords[:10]}')


Number of stopwords: 1942
First 10 stopwords: ['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy']


Tiếp theo, chúng ta phân tách dữ liệu văn bản của mình thành đơn vị nhỏ nhất được gọi là "tokens" hoặc "words". Việc phân tích các đoạn văn dài khá khó khăn nên trước tiên chúng ta tách các đoạn văn thành các dòng riêng biệt sau đó các dòng tách thành các từ và xóa stop words:

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import re
lemmatizer = WordNetLemmatizer()
def clean_para(content):
    """Làm sạch văn bản bằng cách loại bỏ dấu câu, chuyển đổi thành chữ thường và loại bỏ stopwords."""
    # Kiểm tra nếu van_ban không phải là chuỗi, thì chuyển thành chuỗi
    if not isinstance(content, str):
        content = ' '.join(content)
    # Chuyển đổi thành chữ thường
    content = word_tokenize(content.lower())
    # Loại bỏ stopwords
    vocab = [lemmatizer.lemmatize(word) for word in content if word not in stopwords]
    return vocab

In [ ]:
data['content_cleaned'] = data['content'].apply(clean_para)
print(data[['content_cleaned']])

                                       content_cleaned
0    [bloomberg, billionaire, index, ,, jensen, hua...
1    [giao, dịch, giằng, co, phiên, ,, phiên, chiều...
2    [kết, phiên, giao, dịch, 22/5, (, mỹ, ), ,, cổ...
3    [rung, lắc, phiên, tiếp, tục, chỉnh, đầu, phiê...
4    [đỗ, quang, vinh, -, phó, chủ, tịch, hội, đồng...
..                                                 ...
575  [thị, trường, giao, dịch, căng, thẳng, mệt, mỏ...
576  [công, ty, chứng, khoán, agribank, (, agirseco...
577  [phiên, hôm, (, 7/3, ), ,, vn-index, tiếp, tục...
578  [mặc, lực, chiều, dâng, đà, thu, hẹp, yếu, dần...
579  [công, ty, chứng, khoán, agribank, (, agriseco...

[580 rows x 1 columns]


**Xây dựng mô hình LSA**

In [19]:
from sklearn.model_selection import train_test_split
# Chia dữ liệu thành tập huấn luyện, xác thực và tập kiểm tra 70-15-15
text_train, text_val_test = train_test_split(data, test_size=0.3, random_state=42)
text_val, text_test = train_test_split(text_val_test, test_size=0.5, random_state=42)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Chuyển đổi văn bản thành ma trận TF-IDF
vectorizer = TfidfVectorizer(tokenizer=clean_para)
train_vectors = vectorizer.fit_transform(text_train['content'])

# Áp dụng LSA
n_components = 100  # Số chiều của không gian LSA 
lsa = TruncatedSVD(n_components=n_components)
train_lsa = lsa.fit_transform(train_vectors)

***Tóm tắt văn bản***

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

# Định nghĩa pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svd', TruncatedSVD()),
])

# Định nghĩa lưới tham số
param_grid = {
    'tfidf__min_df': [1, 2],  # Try different min_df values
    'tfidf__max_df': [0.8, 0.9],  # Try different max_df values
    'svd__n_components': [2, 3, 4]  # Start with lower values for n_components
}
# Tạo đối tượng GridSearchCV. Xác định score.
def my_scorer(estimator, X):
    X_reduced = estimator.transform(X)
    return cosine_similarity(X_reduced).mean()

# Thực hiện tìm kiếm lưới
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=my_scorer, verbose=0)
grid_search.fit(text_train['content'])

# In ra tham số tốt nhất
print(grid_search.best_params_)

{'svd__n_components': 2, 'tfidf__max_df': 0.8, 'tfidf__min_df': 1}


In [46]:
def summarize_text(text, vectorizer, svd_model, train_lsa, n_sentences=3):
    text = '\n'.join([line for line in text.split('\n') if line.strip()])
    text_vector = vectorizer.transform([text])
    text_lsa = svd_model.transform(text_vector)
    
    # Tính toán độ tương đồng cosine giữa các câu và văn bản gốc
    similarities = cosine_similarity(text_lsa, train_lsa)[0]
    
    # Chọn n câu có độ tương đồng cao nhất, đảm bảo không vượt quá số lượng câu có sẵn
    num_sentences_to_select = min(n_sentences, len(sent_tokenize(text)))  # Calculate the actual number of sentences to select
    top_indices = similarities.argsort()[-num_sentences_to_select:][::-1]  # Select the top indices
    top_indices = sorted(top_indices)

    # Tạo bản tóm tắt
    summary = ' '.join([sent_tokenize(text)[i] for i in top_indices if i < len(sent_tokenize(text))]) # Check if index is valid
    return summary

In [47]:
# Giả sử bộ tham số tốt nhất tìm được là:
best_params = grid_search.best_params_

# Huấn luyện mô hình với bộ tham số tốt nhất trên tập dữ liệu huấn luyện
best_tfidf = TfidfVectorizer(min_df=best_params['tfidf__min_df'], max_df=best_params['tfidf__max_df'])
best_svd = TruncatedSVD(n_components=best_params['svd__n_components'])

text_train_vector = best_tfidf.fit_transform(text_train['content'])
train_data_lsa = best_svd.fit_transform(text_train_vector)

# Kiểm tra hiệu suất của mô hình trên tập dữ liệu kiểm định và kiểm tra
text_val_vector = best_tfidf.transform(text_val['content'])
val_data_lsa = best_svd.transform(text_val_vector)

text_test_vector = best_tfidf.transform(text_test['content'])
test_data_lsa = best_svd.transform(text_test_vector)

# Sử dụng hàm summarize_text để tạo tóm tắt cho các đoạn văn trong tập dữ liệu kiểm định và kiểm tra
text_val['summaries'] = [summarize_text(text, best_tfidf, best_svd, train_data_lsa) for text in text_val['content']] 
text_test['summaries'] = [summarize_text(text, best_tfidf, best_svd, train_data_lsa) for text in text_test['content']]


In [48]:
text_val['summaries']

296                                                     
463                                                     
79                                                      
543                                                     
566                                                     
                             ...                        
155                                                     
441                                                     
414                                                     
271    Trong khi đó, tốc độ tăng trưởng nợ xấu và tỷ ...
84     Tiền đổ vào HoSE đạt 11.661,5 tỷ đồng, khối lư...
Name: summaries, Length: 87, dtype: object

In [49]:
text_test['summaries']

144    Để được xét duyệt nâng hạng, chứng khoán Việt ...
220                                                     
553                                                     
92                                                      
573                                                     
                             ...                        
423                                                     
348                                                     
153                                                     
551    Tổng số mã giảm trên cả 3 sàn là 523 mã so với...
188                                                     
Name: summaries, Length: 87, dtype: object